# Day 23 Unstable Diffusion

Approach: hold coordinates of each elf as x,y tuple


In [101]:
from itertools import *
from collections import defaultdict

#all 8 directions we need to consider. Note, line load in from top, so +1 is south
N = (0,-1)
NE = (1,-1)
NW = (-1,-1)
E = (1,0)
W = (-1,0)
S = (0,1)
SE = (1,1)
SW = (-1,1)
directions = [N,NE,NW,E,W,S,SE,SW]



def addCoord(a:tuple[int,int], b:tuple[int,int]):
    return (a[0] + b[0], a[1] + b[1])
 
class Game:
    def __init__(self, input:str):
        self.elves = set()
        for y, line in enumerate(input.splitlines()):
            for x, c in enumerate(line):
                if c == '#':
                    self.elves.add((x,y))
                x += 1
        self.rounds = 0

        #a lookup of [([cells that need to be empty],movedirection)]
        self.moverules = [   ([N, NE, NW],N),
                ([S, SE, SW],S),
                ([W,NW,SW],W),
                ([E,NE,SE],E) ]

    def runRounds(self,numRounds:int):
        for _ in range(numRounds):
            self.runRound()

    def runUntilComplete(self)->int:
        while self.runRound():
            continue
        return self.rounds

    def runRound(self)->bool:
        #return true if any elf moved
        self.rounds += 1
        output = False
        proposedPositions = defaultdict(lambda:[])
        for elf in self.elves:
            #if no elves in any of the directions, do nothing
            allDirections =  set([addCoord(elf,d) for d in directions])
            if len(self.elves.intersection(allDirections)) == 0:
                #elf does nothing
                #BUT... could it block another elf from moving here? ... doens't change output
                proposedPositions[elf].append(elf)
                continue
            else:
                for test, direction in self.moverules:
                    t = set([addCoord(elf,v) for v in test])
                    if len(self.elves.intersection(t)) == 0:
                        pos = addCoord(elf,direction)
                        proposedPositions[pos].append(elf)
                        break
                else:
                    #no rules matched... so don't move
                    proposedPositions[elf].append(elf)
        #we know know all the proposed moves
        for pos, elf in proposedPositions.items():
            if len(elf) == 1:
                if elf[0] != pos:
                    self.elves.remove(elf[0])
                    self.elves.add(pos)
                    output = True
        #rotate the rule moves
        self.moverules.append(self.moverules.pop(0))
        return output
            
    def emptyTiles(self)->int:
        #number of empty tiles in smallest bounding box
        x = [e[0] for e in self.elves]
        y = [e[1] for e in self.elves]
        return (max(x) - min(x) + 1) * (max(y) - min(y) + 1) - len(self.elves)


In [102]:
testInput = """..............
..............
.......#......
.....###.#....
...#...#.#....
....#...##....
...#.###......
...##.#.##....
....#..#......
..............
..............
.............."""

tg = Game(testInput)
tg.runRounds(10)
print(tg.emptyTiles())
#part 2
print(tg.runUntilComplete())


110
20


In [104]:
puzzleInput = open('day23input.txt').read().strip()
g = Game(puzzleInput)
g.runRounds(10)
print(g.emptyTiles())
print(g.runUntilComplete())


3931
944
